<a href="https://colab.research.google.com/github/deacs11/CrewAI_Contract_Clause_Risk_Assessment/blob/main/CrewAI_Contract_Clause_Risk_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 1. Install necessary libraries
# Installs crewai, tools, OpenAI client, Colab module, and PyMuPDF for PDF handling.
!pip install crewai crewai-tools langchain-openai google-colab pymupdf -q

print("Library installation completed! (PyMuPDF added)")

Library installation completed! (PyMuPDF added)


In [ ]:
# @title 2. Import modules and configure API Keys from Secrets
import os
from google.colab import userdata # To read Colab secrets
from crewai import Agent, Task, Crew, Process
# No external tools needed for this basic version (Serper, etc.)
# from crewai_tools import SerperDevTool, WebsiteSearchTool
from langchain_openai import ChatOpenAI
import re # Regular expressions might be useful for basic parsing

# --- API KEY CONFIGURATION FROM COLAB SECRETS ---
print("Attempting to load API Keys from Secrets...")
try:
    # Reads the OpenAI key from the secret 'OPENAI_API_KEY'
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    print("-> OpenAI API Key environment variable set successfully.")
except Exception as e:
    print(f"!!! Error loading OpenAI API Key from Secrets: {e}")
    print("!!! Make sure you have set 'OPENAI_API_KEY' correctly in the Colab Secrets panel")
    print("!!! and enabled 'Notebook access'.")

# Debug block to verify the key
print("-" * 20)
retrieved_key = os.environ.get("OPENAI_API_KEY")
if retrieved_key:
    print(f"OpenAI Key FOUND in environment: '{retrieved_key[:5]}...{retrieved_key[-4:]}'")
else:
    print("!!! OpenAI Key NOT FOUND in environment.")
    print("!!! Check Colab Secrets and re-run this cell.")
print("-" * 20)

Attempting to load API Keys from Secrets...
-> OpenAI API Key environment variable set successfully.
--------------------
OpenAI Key FOUND in environment: 'sk-pr...DKkA'
--------------------


In [ ]:
# @title 3. Upload contract & define text (Handles PDF and TXT)

from google.colab import files
import os
import fitz  # PyMuPDF library for PDF handling

# --- Instructions for User ---
print("Please upload your contract file using the Colab interface:")
print("1. Click the 'Folder' icon in the left sidebar.")
print("2. Click the 'Upload to session storage' button (icon with an upward arrow).")
print("3. Select your contract file (PDF or TXT format supported).")
print("4. After uploading, wait for it to appear in the file list.")
print("5. **Important:** Right-click the uploaded file and select 'Copy path'.")
print("6. Paste the copied path into the 'uploaded_file_path' field below and run this cell.")
print("-" * 20)

# --- Colab Form Field for File Path ---
uploaded_file_path = "/content/SAMPLE CONTRACT START.pdf"  # @param {type:"string"}

contract_text = "" # Initialize variable

print(f"Attempting to load contract from path: {uploaded_file_path}")

if not os.path.exists(uploaded_file_path):
    print(f"!!! ERROR: File not found at the specified path: '{uploaded_file_path}'")
    print("!!! Please ensure the file is uploaded and the path is copied correctly.")
    # Option: You might want to stop execution here or use a default fallback
    # raise FileNotFoundError(f"Contract file not found at {uploaded_file_path}.")
    print("\n--- No file loaded. Cannot proceed without contract text or fallback. ---")

else:
    try:
        file_extension = os.path.splitext(uploaded_file_path)[1].lower()

        if file_extension == ".pdf":
            print("Detected PDF file. Attempting text extraction with PyMuPDF...")
            doc = fitz.open(uploaded_file_path)
            extracted_texts = []
            for page_num in range(len(doc)):
                page = doc.load_page(page_num)
                extracted_texts.append(page.get_text())
            doc.close()
            contract_text = "\n".join(extracted_texts) # Join text from all pages
            print(f"Successfully extracted text from {len(extracted_texts)} PDF pages.")

        elif file_extension == ".txt":
            print("Detected TXT file. Reading as plain text...")
            with open(uploaded_file_path, 'r', encoding='utf-8') as f:
                contract_text = f.read()
            print("Successfully loaded text from TXT file.")

        else:
            print(f"!!! WARNING: Unsupported file extension '{file_extension}'. Only .pdf and .txt are directly supported by this script.")
            print("--- Attempting to read as plain text anyway (might fail or produce garbage)... ---")
            try:
                 with open(uploaded_file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    contract_text = f.read()
                 print("Read file as plain text, but content interpretation might be incorrect.")
            except Exception as text_read_err:
                 print(f"--- Failed to read as plain text: {text_read_err} ---")
                 contract_text = "" # Ensure it's empty if unsupported and failed reading

        # Display a preview if text was extracted/read
        if contract_text:
            print("\n--- Start of Loaded/Extracted Contract Text (Preview) ---")
            print(contract_text[:1000].strip() + "...") # Show first 1000 characters
            print("--- End of Preview ---")
            if len(contract_text) < 100: # Arbitrary short length check
                 print("--- Warning: Extracted text seems very short. Check PDF content quality (e.g., scanned image?). ---")
        elif file_extension in ['.pdf', '.txt']:
             print("!!! WARNING: The file was processed but no text was extracted/read. The file might be empty, password-protected, image-based, or corrupted.")


    except Exception as e:
        print(f"!!! An error occurred while processing the file: {e}")
        contract_text = "" # Ensure text is empty on error


# --- !!! IMPORTANT DISCLAIMER !!! ---
# (Keep the disclaimer code here)
print("\n" + "*"*70)
print("!!! DISCLAIMER: Contract Analysis Assistant !!!")
print("This tool provides automated analysis to *assist* in identifying potential")
print("areas of interest or risk in contract text based on common patterns.")
print("It DOES NOT provide legal advice. The output is generated by an AI")
print("and may contain errors, omissions, or misinterpretations (especially with complex PDF layouts).")
print("All findings MUST be reviewed by a qualified legal professional")
print("before making any decisions or taking any action.")
print("Do not rely solely on this tool for legal assessment.")
print("*"*70)

# Final check if contract_text is available
if not contract_text:
    print("\n!!! CRITICAL ERROR: No contract text available for analysis. File loading/extraction might have failed.")
    # Optionally raise an error
    # raise ValueError("Cannot proceed without contract text.")
else:
    print("\nContract text is ready for analysis by the Crew.")

Please upload your contract file using the Colab interface:
1. Click the 'Folder' icon in the left sidebar.
2. Click the 'Upload to session storage' button (icon with an upward arrow).
3. Select your contract file (PDF or TXT format supported).
4. After uploading, wait for it to appear in the file list.
5. **Important:** Right-click the uploaded file and select 'Copy path'.
6. Paste the copied path into the 'uploaded_file_path' field below and run this cell.
--------------------
Attempting to load contract from path: /content/SAMPLE CONTRACT START.pdf
Detected PDF file. Attempting text extraction with PyMuPDF...
Successfully extracted text from 13 PDF pages.

--- Start of Loaded/Extracted Contract Text (Preview) ---
=== SAMPLE CONTRACT START === 
 
MASTER SERVICES AGREEMENT 
 
This Master Services Agreement ("MSA") is made and entered into as of this 1st day of October, 2024 
("Effective Date"), by and between: 
 
Innovate Solutions Inc., a Delaware corporation with its principal place

In [ ]:
# @title 4. Select LLM and initialize Tools

# --- CHOOSE THE LANGUAGE MODEL (LLM) ---
try:
    # GPT-4 is highly recommended for the nuance required in legal text analysis
    llm = ChatOpenAI(model="gpt-4-turbo")
    # llm = ChatOpenAI(model="gpt-3.5-turbo") # Might struggle significantly with legal nuance
    print(f"LLM ({llm.model_name}) initialized successfully.")
except Exception as e:
    print(f"!!! Error initializing ChatOpenAI: {e}")
    print("!!! Verify OpenAI API key.")
    raise

# --- INITIALIZE TOOLS ---
# For this version, we assume the contract text is passed directly.
# No external search or web reading tools are needed by the core agents.
available_tools = []
print("No external tools initialized for this crew.")

LLM (gpt-4-turbo) initialized successfully.
No external tools initialized for this crew.


In [ ]:
# @title 5. Define agents for Contract Analysis Crew

print("Defining Contract Analysis Agents...")

if 'llm' not in locals() or llm is None:
     raise ValueError("LLM not initialized.")

# --- Agent 1: Contract Parser ---
# Basic parsing attempt. A more robust solution might involve dedicated libraries or regex.
contract_parser = Agent(
    role='Legal Document Structure Analyst',
    goal='Segment the provided contract text into individually identifiable clauses or sections. Preserve numbering if present. Output a list where each item represents a distinct clause or paragraph.',
    backstory=(
        "You are meticulous at analyzing document structure. You identify distinct paragraphs or numbered items as separate clauses. "
        "Your goal is to break down the contract into its core components for easier analysis by other specialists. Handle potential variations in numbering or formatting."
    ),
    tools=[],
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=3
)
print("- Agent 'contract_parser' defined.")

# --- Agent 2: Clause classifier (optional but helpful) ---
# This helps downstream agents focus, but adds a step. Can be removed if needed.
clause_classifier = Agent(
    role='Legal Clause Taxonomy Expert',
    goal='For each segmented clause provided, assign a likely category based on its content (e.g., Services, Term, Payment, Confidentiality, Warranty, Liability, Indemnification, Termination, Governing Law, Miscellaneous).',
    backstory=(
        "You have a deep understanding of common contract structures and the purpose of different clauses. "
        "Based on keywords and context, you can accurately categorize most standard contract provisions. "
        "Acknowledge if a clause seems miscellaneous or hard to classify."
    ),
    tools=[],
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=5
)
print("- Agent 'clause_classifier' defined.")

# --- Agent 3: Risk Pattern Detector ---
risk_pattern_detector = Agent(
    role='Contract Risk Spotter',
    goal='Scan each provided contract clause text. Identify and flag clauses containing specific high-risk patterns or keywords commonly associated with unfavorable terms. Focus on patterns like: unlimited liability, broad indemnification obligations imposed ON THE CLIENT, weak warranties (e.g., extensive disclaimers), ambiguous or one-sided termination rights, automatic renewals without clear opt-out, non-standard governing law/jurisdiction choices, overly broad confidentiality obligations.',
    backstory=(
        "You are trained to recognize red flags in contract language based on common legal and business risk concerns. "
        "You meticulously compare clause text against a known set of potentially problematic phrases and concepts. "
        "You don't interpret the overall fairness, just flag the presence of specific predefined risk patterns."
    ),
    tools=[],
    llm=llm, # This agent heavily relies on the LLM's pattern matching based on the goal description
    allow_delegation=False,
    verbose=True,
    max_iter=7
)
print("- Agent 'risk_pattern_detector' defined.")

# --- Agent 4: Ambiguity Identifier ---
ambiguity_identifier = Agent(
    role='Clarity and Precision Analyst',
    goal='Review each contract clause for ambiguous language, undefined critical terms (e.g., "reasonable efforts", "material breach" without definition), potentially contradictory statements, or overly broad phrasing that could lead to future disputes or misinterpretations. Flag clauses requiring clarification.',
    backstory=(
        "You focus intensely on linguistic precision and clarity in legal documents. You hunt for words or phrases that lack specific definition "
        "where one might be needed, or sentences structured in a way that allows for multiple interpretations. Your aim is to flag areas needing refinement for certainty."
    ),
    tools=[],
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=5
)
print("- Agent 'ambiguity_identifier' defined.")

# --- Agent 5: Review Brief Generator ---
review_brief_generator = Agent(
    role='Legal Review Summarizer',
    goal='Consolidate all flagged clauses identified by the Risk Pattern Detector and the Ambiguity Identifier into a single, structured "Review Brief". For each flagged item, clearly state the clause number (or reference), the clause text, and the specific reason(s) it was flagged (e.g., "Risk Pattern: Unlimited Liability", "Ambiguity: Undefined term \'material\'"). Organize the brief for efficient review by a legal professional.',
    backstory=(
        "You excel at synthesizing analytical findings into clear, actionable summaries for busy professionals. "
        "You organize flagged items logically, provide necessary context (clause text and reason), and ensure the output is easy to scan and understand. "
        "The goal is to facilitate a focused human review."
    ),
    tools=[],
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=3
)
print("- Agent 'review_brief_generator' defined.")

print("All contract analysis agents defined.")

Defining Contract Analysis Agents...
- Agent 'contract_parser' defined.
- Agent 'clause_classifier' defined.
- Agent 'risk_pattern_detector' defined.
- Agent 'ambiguity_identifier' defined.
- Agent 'review_brief_generator' defined.
All contract analysis agents defined.


In [ ]:
# @title 6. Define Tasks for Contract Analysis Crew

print("Defining Contract Analysis Tasks...")

# Verify agents are defined
if 'contract_parser' not in locals() or \
   'clause_classifier' not in locals() or \
   'risk_pattern_detector' not in locals() or \
   'ambiguity_identifier' not in locals() or \
   'review_brief_generator' not in locals():
    raise ValueError("One or more required agents are not defined.")

# --- Task 1: Parse Contract into Clauses ---
task_parse_contract = Task(
    description=(
        f"Process the following contract text provided in the initial input:\n---\n{contract_text}\n---\n"
        f"Segment this text into a list of distinct clauses or sections. Attempt to preserve any existing numbering (e.g., '1.', 'Section 2.1')."
        f"Output should be a structured representation (like a numbered list or list of strings) of these segmented clauses."
    ),
    agent=contract_parser,
    expected_output=(
        "A list containing the text of each identified clause or section from the input contract. "
        "Example: ['1. SERVICES. Provider agrees...', '2. TERM. This Agreement shall...', ...]"
    )
)
print("- Task 'task_parse_contract' defined.")

# --- Task 2: Classify Clauses ---
task_classify_clauses = Task(
    description=(
        "Take the list of segmented contract clauses (provided from the previous task's context). "
        "For each clause, determine its likely primary category (e.g., Services, Term, Payment, Confidentiality, Warranty, Limitation of Liability, Indemnification, Termination, Governing Law, Entire Agreement/Miscellaneous). "
        "Present the output as a list where each item contains the original clause text and its assigned category."
    ),
    agent=clause_classifier,
    expected_output=(
        "A list, where each element corresponds to a clause and includes the clause text and its assigned category. "
        "Example: [{'clause_text': '1. SERVICES...', 'category': 'Services'}, {'clause_text': '6. LIMITATION OF LIABILITY...', 'category': 'Limitation of Liability'}, ...]"
    ),
    context=[task_parse_contract] # Needs the segmented clauses
)
print("- Task 'task_classify_clauses' defined.")

# --- Task 3: Detect Risk Patterns ---
task_detect_risks = Task(
    description=(
        "Review the list of classified contract clauses (provided in context). "
        "Scan the text of *each* clause specifically for the presence of predefined high-risk patterns or keywords (as defined in the Risk Spotter agent's goal, e.g., unlimited liability, broad client indemnification, weak warranties, ambiguous termination, auto-renewal issues, non-standard jurisdiction). "
        "Output a list containing ONLY the clauses that were flagged, including the clause text and the specific risk pattern(s) identified for each flagged clause."
    ),
    agent=risk_pattern_detector,
    expected_output=(
        "A list containing only the clauses identified as potentially risky. Each item in the list should include the clause text and the specific reason(s)/pattern(s) why it was flagged. If no risks are found, the list should be empty. "
        "Example: [{'clause_text': '6. LIMITATION OF LIABILITY...', 'risk_flag': 'Broad disclaimer of implied warranties'}, {'clause_text': '7. INDEMNIFICATION...', 'risk_flag': 'Client indemnifies Provider broadly'}]"
    ),
    context=[task_classify_clauses] # Needs the classified clauses (provides context and text)
)
print("- Task 'task_detect_risks' defined.")

# --- Task 4: Identify Ambiguities ---
task_identify_ambiguities = Task(
    description=(
        "Review the list of classified contract clauses (provided in context). "
        "Analyze the text of *each* clause for potential ambiguities, undefined critical terms, vague language, or contradictions that might require clarification. "
        "Output a list containing ONLY the clauses that were flagged for ambiguity, including the clause text and a brief explanation of the ambiguity identified in each flagged clause."
    ),
    agent=ambiguity_identifier,
    expected_output=(
        "A list containing only the clauses identified as potentially ambiguous. Each item should include the clause text and a brief note explaining the source of ambiguity. If no ambiguities are found, the list should be empty. "
        "Example: [{'clause_text': '1. SERVICES... Provider shall determine the method...', 'ambiguity_flag': 'Term \"method, details, and means\" is vague'}, {'clause_text': '8. TERMINATION... materially breaches...', 'ambiguity_flag': 'Term \"materially breaches\" is undefined'}]"
    ),
    context=[task_classify_clauses] # Needs the classified clauses
)
print("- Task 'task_identify_ambiguities' defined.")

# --- Task 5: Generate Review Brief ---
task_generate_brief = Task(
    description=(
        "Consolidate the findings from the risk detection task and the ambiguity identification task (outputs provided in context) into a structured 'Contract Review Brief'. "
        "The brief should have two main sections: 'Potential Risk Flags' and 'Potential Ambiguities/Clarifications Needed'. "
        "Under each section, list the items identified previously, including the original clause text (or a clear reference like clause number if consistently available from parsing) and the specific reason it was flagged (risk pattern or ambiguity explanation). "
        "Format the output clearly using Markdown for easy reading by a legal professional. Include the disclaimer about this being an assistant tool."
    ),
    agent=review_brief_generator,
    expected_output=(
        "A well-formatted Markdown document titled 'Contract Review Brief (AI Assisted)'. "
        "It must include the disclaimer. "
        "It should contain two distinct sections: 'Potential Risk Flags' and 'Potential Ambiguities/Clarifications Needed'. "
        "Each section should list the relevant flagged clauses with their text and the reason for flagging. "
        "The output should be ready for human review."
    ),
    # Needs the outputs from BOTH the risk detection and ambiguity identification tasks
    context=[task_detect_risks, task_identify_ambiguities]
)
print("- Task 'task_generate_brief' defined.")

print("All contract analysis tasks defined.")

Defining Contract Analysis Tasks...
- Task 'task_parse_contract' defined.
- Task 'task_classify_clauses' defined.
- Task 'task_detect_risks' defined.
- Task 'task_identify_ambiguities' defined.
- Task 'task_generate_brief' defined.
All contract analysis tasks defined.


In [ ]:
# @title 7. Create and Run Contract Analysis Crew

print("Creating the Contract Analysis Crew...")

# Verify all components are ready
if 'contract_parser' not in locals() or \
   'clause_classifier' not in locals() or \
   'risk_pattern_detector' not in locals() or \
   'ambiguity_identifier' not in locals() or \
   'review_brief_generator' not in locals() or \
   'task_parse_contract' not in locals() or \
   'task_classify_clauses' not in locals() or \
   'task_detect_risks' not in locals() or \
   'task_identify_ambiguities' not in locals() or \
   'task_generate_brief' not in locals():
    raise ValueError("Missing agents or tasks needed to create the Contract Analysis Crew.")

# Assemble the Crew
contract_analysis_crew = Crew(
    agents=[contract_parser, clause_classifier, risk_pattern_detector, ambiguity_identifier, review_brief_generator],
    tasks=[task_parse_contract, task_classify_clauses, task_detect_risks, task_identify_ambiguities, task_generate_brief],
    process=Process.sequential,
    memory=True, # Memory helps pass context between these detailed tasks
    cache=True, # Cache might not be very useful here unless tools were used
    verbose=True
)

print("Contract Analysis Crew created. Starting the process with kickoff()...")

# Define initial inputs - only the contract text is needed here
# as it's explicitly referenced in the first task's description.
crew_inputs = {
    'contract_full_text': contract_text # Provide a key for clarity, even if unused directly by kickoff
}

# Kick off the crew's work
# This will perform multiple analysis steps on the contract text
result = contract_analysis_crew.kickoff(inputs=crew_inputs)

print("\n\n*****************************************")
print("   CONTRACT ANALYSIS CREW FINISHED WORK!   ")
print("*****************************************")

Creating the Contract Analysis Crew...
Contract Analysis Crew created. Starting the process with kickoff()...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: e8a41a47-890e-4129-8e38-934b6e740b2b                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 8e07eb72-b56b-433e-a4ca-3c9e109e1f57
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 8e07eb72-b56b-433e-a4ca-3c9e109e1f57
       Status: Executing Task...
    └── 🤖 Agent: Legal Document Structure Analyst
            Status: In Progress

# Agent: Legal Document Structure Analyst
## Task: Process the following contract text provided in the initial input:
---
=== SAMPLE CONTRACT START === 
 
MASTER SERVICES AGREEMENT 
 
This Master Services Agreement ("MSA") is made and entered into as of this 1st day of October, 2024 
("Effective Date"), by and between: 
 
Innovate Solutions Inc., a Delaware corporation with its principal place of business at 123 Tech Avenue, Suite 
400, Anytown, CA 90210 ("Provider"), 
 
and 
 
Client Corp., a California corporation with its principal place of business at 456 Business Drive, Metropolis, 
CA 90211 ("Client"). 
 
(Provider and Client are sometimes referred to herein individually as a "Party" and collectively as the 
"Parties"). 
 
RECITALS 
 
WHEREAS, Provider is in the business of providing consulting and development services in the ﬁeld of 
software engineering and digital transformation; 
 
WHEREAS, Client desires to engage Provider from time to time to provide certain services as mor

...

🚀 Crew: crew
├── 📋 Task: 8e07eb72-b56b-433e-a4ca-3c9e109e1f57
│      Assigned to: Legal Document Structure Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Legal Document Structure Analyst
│           Status: ✅ Completed
├── 📋 Task: 850c1353-b009-4ff2-ba5e-802f6a5d97a0
│      Assigned to: Legal Clause Taxonomy Expert
│      Status: ✅ Completed
│   └── 🤖 Agent: Legal Clause Taxonomy Expert
│           Status: ✅ Completed
├── 📋 Task: b5f14555-8a22-4534-8874-a8dc97094987
│      Assigned to: Contract Risk Spotter
│      Status: ✅ Completed
│   └── 🤖 Agent: Contract Risk Spotter
│           Status: ✅ Completed
├── 📋 Task: fb2e8fc2-26bd-4053-a4f3-1bb13da1e2e1
│      Assigned to: Clarity and Precision Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Clarity and Precision Analyst
│           Status: ✅ Completed
└── 📋 Task: 7db1b921-89ce-4c34-93b5-4948ad01f7cf
       Status: Executing Task...
    └── 🤖 Agent: Legal Review Summarizer
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 8e07eb72-b56b-433e-a4ca-3c9e109e1f57
│      Assigned to: Legal Document Structure Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Legal Document Structure Analyst
│           Status: ✅ Completed
├── 📋 Task: 850c1353-b009-4ff2-ba5e-802f6a5d97a0
│      Assigned to: Legal Clause Taxonomy Expert
│      Status: ✅ Completed
│   └── 🤖 Agent: Legal Clause Taxonomy Expert
│           Status: ✅ Completed
├── 📋 Task: b5f14555-8a22-4534-8874-a8dc97094987
│      Assigned to: Contract Risk Spotter
│      Status: ✅ Completed
│   └── 🤖 Agent: Contract Risk Spotter
│           Status: ✅ Completed
├── 📋 Task: fb2e8fc2-26bd-4053-a4f3-1bb13da1e2e1
│      Assigned to: Clarity and Precision Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Clarity and Precision Analyst
│           Status: ✅ Completed
└── 📋 Task: 7db1b921-89ce-4c34-93b5-4948ad01f7cf
       Assigned to: Legal Review Summarizer
       Status: ✅ Completed
    └── 🤖 Agent: Legal Review Summarizer
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 7db1b921-89ce-4c34-93b5-4948ad01f7cf                                                                     │
│  Agent: Legal Review Summarizer                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: e8a41a47-890e-4129-8e38-934b6e740b2b                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



*****************************************
   CONTRACT ANALYSIS CREW FINISHED WORK!   
*****************************************


In [ ]:
# @title 8. Display final review brief

print("\n\n#####################################")
print("## AI-Assisted Contract Review Brief:")
print("## REMINDER: FOR HUMAN REVIEW ONLY - NOT LEGAL ADVICE")
print("#####################################\n")

if 'result' in locals():
    # Print the final output from the 'review_brief_generator' agent
    print(result)
else:
    print("!!! The 'result' variable is not defined. Ensure Cell 7 executed correctly.")

# Add final disclaimer again for safety
print("\n\n"+"*"*70)
print("!!! IMPORTANT: The above output is AI-generated assistance.")
print("!!! It MUST be reviewed thoroughly by a qualified legal professional.")
print("!!! Do not rely on this output for legal decisions.")
print("*"*70)



#####################################
## AI-Assisted Contract Review Brief:
## REMINDER: FOR HUMAN REVIEW ONLY - NOT LEGAL ADVICE
#####################################

# Contract Review Brief (AI Assisted)

**Disclaimer:** This document is prepared by an AI tool to assist in the initial review of the contract clauses. It identifies potential risks and ambiguities that may require further human analysis and should not be used as a substitute for professional legal advice.

---

## Potential Risk Flags

1. **Clause 4.2: Automatic Renewal**
   - **Text:** "Upon expiration of the Initial Term, this MSA shall automatically renew for successive one (1) year periods, unless either Party provides written notice of non-renewal at least sixty (60) days prior to the end of the then-current term."
   - **Flagged Reason:** Automatic renewal without clear opt-out mechanisms increases the risk of inadvertently extending a contract that might no longer serve the interests of one or both parties.

2